# Elastic Critical Load Analysis

This notebook performs a structural analysis to determine the critical buckling load of a 3D beam using the Direct Stiffness Method (DSM) and eigenvalue analysis. The beam is modeled with one fixed end and one free end, subjected to a compressive load. We will compute and display:
- **Displacements** at the free node.
- **Internal forces and moments** for the element.
- **Critical load factor** for buckling.

### Imports
We import necessary libraries for numerical computations, plotting, and custom modules for DSM and geometric stiffness calculations.

In [ ]:
import numpy as np
import Direct_Stiffness_Method as dsm
import functions as fu
import Elastic_Critical_Solver as ecs

### Define Nodes
We define two nodes:
- **Node 0**: Fixed at the origin (0, 0, 0) with all 6 DOFs (degrees of freedom) constrained.
- **Node 1**: Free at (30, 40, 0) with no boundary conditions, allowing full movement and rotation.

Each node is represented as a dictionary with coordinates and boundary conditions (`bc`).

In [ ]:
nodes = [
    {'node_id': 0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'bc': [True, True, True, True, True, True]},  # Fixed
    {'node_id': 1, 'x': 30.0, 'y': 40.0, 'z': 0.0, 'bc': [False, False, False, False, False, False]}  # Free
]

### Material and Section Properties
Define the properties of the beam:
- **A**: Cross-sectional area (π m², assuming a circular section).
- **E**: Young's modulus (1000 Pa, low for illustrative purposes).
- **nu**: Poisson's ratio (0.3).
- **Iy, Iz**: Moments of inertia about y and z axes (π/4 m⁴).
- **J**: Torsion constant (π/2 m⁴).

These values are used to compute stiffness matrices.

In [ ]:
A = np.pi       # Cross-sectional area (m²)
E = 1000        # Young's modulus (Pa)
nu = 0.3        # Poisson's ratio
Iy = np.pi / 4  # Moment of inertia about y (m⁴)
Iz = np.pi / 4  # Moment of inertia about z (m⁴)
J = np.pi / 2   # Torsion constant (m⁴)

### Create Element
We create a single beam element connecting Node 0 to Node 1 using the `create_element` function from the Direct Stiffness Method module. This function computes the local stiffness matrix and rotation matrix based on the node coordinates and material properties.

In [ ]:
element = dsm.create_element(nodes[0], nodes[1], E, nu, A, Iy, Iz, J)
elements = [element]

### Define Loads
A compressive load of 1 N is applied at Node 1, directed along the beam axis:
- **Fx = -3/5 N**, **Fy = -4/5 N** (components based on the beam's orientation, magnitude = 1 N).
- No forces or moments in z-direction or rotational DOFs.

Loads are stored in a dictionary with the node ID as the key.

In [ ]:
loads = {1: np.array([-3/5, -4/5, 0.0, 0.0, 0.0, 0.0])}  # [Fx, Fy, Fz, Mx, My, Mz]

### Initialize Solver
We instantiate the `CriticalLoadSolver` from the `Elastic_Critical_Solver` module. The solver computes the critical buckling load by assembling elastic and geometric stiffness matrices. We set `include_interactions=False` to simplify the geometric stiffness calculation (ignoring interaction terms).

In [ ]:
solver = ecs.CriticalLoadSolver(nodes, elements, loads, include_interactions=False)

### Compute and Display Results
Here, we:
1. **Calculate Displacements**: Using DSM to find the displacement vector under the applied load.
2. **Compute Internal Forces and Moments**: Extract forces and moments for the element (0, 1).
3. **Determine Critical Load Factor**: Solve the eigenvalue problem to find the buckling load factor.
4. **Plot Buckling Mode**: Visualize the deformed shape (optional).

Only the requested outputs are printed:
- Displacements
- Force and Moment
- Critical Load Factor

In [ ]:
# Compute displacements
displacements, _ = dsm.calculate_structure_response(nodes, elements, loads)
print("Displacements:", displacements)

# Compute internal forces and moments
internal_forces = solver.compute_internal_forces_and_moments(displacements)
print("Force and Moment for Element (0,1):", ecs.moments_from_forces(internal_forces, (0, 1)))

# Compute critical load factor
eigenvalues, eigenvectors = solver.compute_eigenvalues()
print("Critical Load Factor:", eigenvalues[0] if eigenvalues.size > 0 else "None")

# Plot the buckling mode shape
solver.plot_mode_shape(eigenvectors[:, 0], scale_factor=0.5)